In [ ]:
! pip install pdfminer.six

In [ ]:
! pip install  seqeval transformers

In [ ]:
import re
import json
import logging
import numpy as np
from tqdm import trange
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


def convert_goldparse(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content'].replace("\n", " ")
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    point = annotation['points'][0]
                    labels = annotation['label']
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']

                        lstrip_diff = len(point_text) - \
                            len(point_text.lstrip())
                        rstrip_diff = len(point_text) - \
                            len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1, label))
            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " +
                          dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data


def get_label(offset, labels):
    if offset[0] == 0 and offset[1] == 0:
        return 'O'
    for label in labels:
        if offset[1] >= label[0] and offset[0] <= label[1]:
            return label[2]
    return 'O'


tags_vals = ["UNKNOWN", "O", "Name", "Degree", "Skills", "College Name", "Email Address",
             "Designation", "Companies worked at", "Graduation Year", "Years of Experience", "Location"]

tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals)}


def process_resume(data, tokenizer, tag2idx, max_len, is_test=False):
    tok = tokenizer.encode_plus(
        data[0], max_length=max_len, pad_to_max_length=True,return_offsets_mapping=True)
    curr_sent = {'orig_labels': [], 'labels': []}

    padding_length = max_len - len(tok['input_ids'])

    if not is_test:
        labels = data[1]['entities']
        labels.reverse()
        for off in tok['offset_mapping']:
            label = get_label(off, labels)
            curr_sent['orig_labels'].append(label)
            curr_sent['labels'].append(tag2idx[label])
        curr_sent['labels'] = curr_sent['labels'] + ([0] * padding_length)

    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent


class ResumeDataset(Dataset):
    def __init__(self, resume, tokenizer, tag2idx, max_len, is_test=False):
        self.resume = resume
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.tag2idx = tag2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.resume)

    def __getitem__(self, idx):
        data = process_resume(
            self.resume[idx], self.tokenizer, self.tag2idx, self.max_len, self.is_test)
        return {
            'input_ids': torch.tensor(data['input_ids'], dtype=torch.long),
            'token_type_ids': torch.tensor(data['token_type_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(data['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(data['labels'], dtype=torch.long),
            'orig_label': data['orig_labels']
        }


def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [
            {"params": [p for n, p in param_optimizer]}]

    return optimizer_grouped_parameters


def get_special_tokens(tokenizer, tag2idx):
    vocab = tokenizer.get_vocab()
    pad_tok = vocab["[PAD]"]
    sep_tok = vocab["[SEP]"]
    cls_tok = vocab["[CLS]"]
    o_lab = tag2idx["O"]

    return pad_tok, sep_tok, cls_tok, o_lab


def annot_confusion_matrix(valid_tags, pred_tags):
    """
    Create an annotated confusion matrix by adding label
    annotations and formatting to sklearn's `confusion_matrix`.
    """

    header = sorted(list(set(valid_tags + pred_tags)))

    matrix = confusion_matrix(valid_tags, pred_tags, labels=header)

    mat_formatted = [header[i] + "\t\t\t" +
                     str(row) for i, row in enumerate(matrix)]
    content = "\t" + " ".join(header) + "\n" + "\n".join(mat_formatted)

    return content


def flat_accuracy(valid_tags, pred_tags):
    return (np.array(valid_tags) == np.array(pred_tags)).mean()


def train_and_val_model(
    model,
    tokenizer,
    optimizer,
    epochs,
    idx2tag,
    tag2idx,
    max_grad_norm,
    device,
    train_dataloader,
    valid_dataloader
):

    pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, tag2idx)

    epoch = 0
    for _ in trange(epochs, desc="Epoch"):
        epoch += 1

        # Training loop
        print("Starting training loop.")
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        for step, batch in enumerate(train_dataloader):
            # Add batch to gpu

            # batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(
                device), b_input_mask.to(device), b_labels.to(device)

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss, tr_logits = outputs[:2]

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            tr_logits = tr_logits.cpu().detach().numpy()
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1)
            tr_batch_labels = tr_label_ids.to("cpu").numpy()
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)

            # Compute training accuracy
            tmp_tr_accuracy = flat_accuracy(tr_batch_labels, tr_batch_preds)
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=max_grad_norm
            )

            # Update parameters
            optimizer.step()
            model.zero_grad()

        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"Train loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")

        """
        Validation loop
        """
        print("Starting validation loop.")

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []

        for batch in valid_dataloader:

            b_input_ids, b_input_mask, b_labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            b_input_ids, b_input_mask, b_labels = b_input_ids.to(
                device), b_input_mask.to(device), b_labels.to(device)

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                tmp_eval_loss, logits = outputs[:2]

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            logits = logits.cpu().detach().numpy()
            label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            preds_mask = preds_mask.cpu().detach().numpy()
            val_batch_preds = np.argmax(logits[preds_mask.squeeze()], axis=1)
            val_batch_labels = label_ids.to("cpu").numpy()
            predictions.extend(val_batch_preds)
            true_labels.extend(val_batch_labels)

            tmp_eval_accuracy = flat_accuracy(
                val_batch_labels, val_batch_preds)

            eval_loss += tmp_eval_loss.mean().item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1

        # Evaluate loss, acc, conf. matrix, and class. report on devset
        
        pred_tags = [idx2tag[i] for i in predictions]
        valid_tags = [idx2tag[i] for i in true_labels]
        cl_report = classification_report(valid_tags, pred_tags)
        conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_steps

        # Report metrics
        print(f"Validation loss: {eval_loss}")
        print(f"Validation Accuracy: {eval_accuracy}")
        print(f"Classification Report:\n {cl_report}")
        print(f"Confusion Matrix:\n {conf_mat}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
from pdfminer.high_level import extract_text



def preprocess_data(data):
    text = extract_text(data)
    text = text.replace("\n", " ")
    text = text.replace("\f", " ")
    return text


def tokenize_resume(text, tokenizer, max_len):
    tok = tokenizer.encode_plus(
        text, max_length=max_len, pad_to_max_length=True, return_offsets_mapping=True)

    curr_sent = dict()

    padding_length = max_len - len(tok['input_ids'])

    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)

    final_data = {
        'input_ids': torch.tensor(curr_sent['input_ids'], dtype=torch.long),
        'token_type_ids': torch.tensor(curr_sent['token_type_ids'], dtype=torch.long),
        'attention_mask': torch.tensor(curr_sent['attention_mask'], dtype=torch.long),
        'offset_mapping': tok['offset_mapping']
    }

    return final_data


tags_vals = ["UNKNOWN", "O", "Name", "Degree", "Skills", "College Name", "Email Address",
             "Designation", "Companies worked at", "Graduation Year", "Years of Experience", "Location"]
idx2tag = {i: t for i, t in enumerate(tags_vals)}
resticted_lables = ["UNKNOWN", "O","Degree", "Skills", "College Name", "Email Address",
             "Designation", "Companies worked at", "Graduation Year", "Years of Experience", "Location"]


def predict(model, tokenizer, idx2tag, device, test_resume, max_len):
    model.eval()
    data = tokenize_resume(test_resume, tokenizer, max_len)
    input_ids, input_mask = data['input_ids'].unsqueeze(
        0), data['attention_mask'].unsqueeze(0)
    labels = torch.tensor([1] * input_ids.size(0),
                          dtype=torch.long).unsqueeze(0)

    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(
            input_ids,
            token_type_ids=None,
            attention_mask=input_mask,
            labels=labels,
        )
        tmp_eval_loss, logits = outputs[:2]

    logits = logits.cpu().detach().numpy()
    label_ids = np.argmax(logits, axis=2)

    entities = []
    for label_id, offset in zip(label_ids[0], data['offset_mapping']):
        curr_id = idx2tag[label_id]
        curr_start = offset[0]
        curr_end = offset[1]
        if curr_id not in resticted_lables:
            if len(entities) > 0 and entities[-1]['entity'] == curr_id and curr_start - entities[-1]['end'] in [0, 1]:
                entities[-1]['end'] = curr_end
            else:
                entities.append(
                    {'entity': curr_id, 'start': curr_start, 'end': curr_end})
    for ent in entities:
        ent['text'] = test_resume[ent['start']:ent['end']]
    return entities


In [ ]:
import argparse
import numpy as np
import torch
from transformers import BertForTokenClassification, BertTokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam

In [ ]:
# parser = argparse.ArgumentParser(description='Train Bert-NER')
# parser.add_argument('-e', type=int, default=5, help='number of epochs')
# parser.add_argument('-o', type=str, default='.',
#                     help='output path to save model state')

# args = parser.parse_args().__dict__

# output_path = args['o']

MAX_LEN = 512
EPOCHS = 20
MAX_GRAD_NORM = 1.0
MODEL_NAME = 'bert-base-uncased'
TOKENIZER = BertTokenizerFast('/content/drive/MyDrive/Resume Data/Copy of vocab.txt', lowercase=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = trim_entity_spans(convert_goldparse('/content/drive/MyDrive/Resume Data/Copy of Resumes.json'))

total = len(data)
train_data, val_data = data[:180], data[180:]

train_d = ResumeDataset(train_data, TOKENIZER, tag2idx, MAX_LEN)
val_d = ResumeDataset(val_data, TOKENIZER, tag2idx, MAX_LEN)

train_sampler = RandomSampler(train_d)
train_dl = DataLoader(train_d, sampler=train_sampler, batch_size=8)

val_dl = DataLoader(val_d, batch_size=8)

model = BertForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=len(tag2idx))
model.to(DEVICE)
optimizer_grouped_parameters = get_hyperparameters(model, True)
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

train_and_val_model(
    model,
    TOKENIZER,
    optimizer,
    EPOCHS,
    idx2tag,
    tag2idx,
    MAX_GRAD_NORM,
    DEVICE,
    train_dl,
    val_dl
)

torch.save(
    {
        "model_state_dict": model.state_dict()
    },
    f'/content/drive/MyDrive/Resume Data/model-state.bin',
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

Starting training loop.
Train loss: 0.9055620457815088
Train accuracy: 0.7834522757238771
Starting validation loop.


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch:   5%|▌         | 1/20 [00:13<04:14, 13.39s/it]

Validation loss: 0.6617066979408264
Validation Accuracy: 0.8316991799301041
Classification Report:
                      precision    recall  f1-score   support

       College Name       0.00      0.00      0.00       208
Companies worked at       0.00      0.00      0.00       236
             Degree       0.00      0.00      0.00       152
        Designation       0.00      0.00      0.00       323
      Email Address       0.67      0.64      0.66      1142
    Graduation Year       0.00      0.00      0.00        20
           Location       0.21      0.03      0.06       175
               Name       1.00      0.38      0.56       190
                  O       0.84      0.98      0.91     12880
             Skills       0.82      0.14      0.24       902
Years of Experience       0.00      0.00      0.00        15

           accuracy                           0.83     16243
          macro avg       0.32      0.20      0.22     16243
       weighted avg       0.78      0.83    

Epoch:  10%|█         | 2/20 [00:26<03:59, 13.28s/it]

...

Validation loss: 0.6656005442142486
Validation Accuracy: 0.8601722273514605
Classification Report:
                      precision    recall  f1-score   support

       College Name       0.60      0.89      0.72       208
Companies worked at       0.49      0.60      0.54       245
             Degree       0.87      0.86      0.86       152
        Designation       0.80      0.55      0.66       314
      Email Address       0.83      0.75      0.78      1142
    Graduation Year       0.17      0.40      0.24        20
           Location       0.67      0.79      0.73       175
               Name       0.98      0.99      0.98       190
                  O       0.94      0.89      0.91     12880
             Skills       0.44      0.79      0.56       902
Years of Experience       0.35      0.40      0.38        15

           accuracy                           0.86     16243
          macro avg       0.65      0.72      0.67     16243
       weighted avg       0.89      0.86    

Epoch:  95%|█████████▌| 19/20 [04:03<00:12, 12.68s/it]

Validation loss: 0.6782584965229035
Validation Accuracy: 0.8606052299969758
Classification Report:
                      precision    recall  f1-score   support

       College Name       0.59      0.88      0.70       208
Companies worked at       0.48      0.54      0.51       236
             Degree       0.74      0.89      0.81       152
        Designation       0.80      0.61      0.69       323
      Email Address       0.79      0.82      0.80      1142
    Graduation Year       0.19      0.40      0.25        20
           Location       0.72      0.81      0.77       175
               Name       0.97      0.98      0.97       190
                  O       0.94      0.88      0.91     12880
             Skills       0.43      0.77      0.55       902
Years of Experience       0.60      0.40      0.48        15

           accuracy                           0.86     16243
          macro avg       0.66      0.73      0.68     16243
       weighted avg       0.89      0.86    

Epoch: 100%|██████████| 20/20 [04:16<00:00, 12.83s/it]

Validation loss: 0.6240813970565796
Validation Accuracy: 0.8770425761358054
Classification Report:
                      precision    recall  f1-score   support

       College Name       0.66      0.81      0.73       208
Companies worked at       0.49      0.47      0.48       245
             Degree       0.87      0.86      0.86       152
        Designation       0.84      0.53      0.65       314
      Email Address       0.80      0.76      0.78      1142
    Graduation Year       0.17      0.35      0.23        20
           Location       0.63      0.81      0.71       175
               Name       0.96      0.98      0.97       190
                  O       0.94      0.91      0.92     12880
             Skills       0.52      0.77      0.62       902
Years of Experience       0.69      0.60      0.64        15

           accuracy                           0.88     16243
          macro avg       0.69      0.71      0.69     16243
       weighted avg       0.89      0.88    

In [ ]:
import io
import argparse
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
 
MAX_LEN = 500
NUM_LABELS = 12
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = 'bert-base-uncased'
STATE_DICT = torch.load("/content/drive/MyDrive/Resume Data/model-state.bin", map_location=DEVICE)
TOKENIZER = BertTokenizerFast("/content/drive/MyDrive/Resume Data/Copy of vocab.txt", lowercase=True)
 
model = BertForTokenClassification.from_pretrained(
 'bert-base-uncased', state_dict=STATE_DICT['model_state_dict'], num_labels=NUM_LABELS)
model.to(DEVICE)


 
data = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/Android-iOS-Flutter/Shariful Islam.pdf', 'rb').read())
resume_text = preprocess_data(data)
entities = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text, MAX_LEN)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities

[{'end': 4, 'entity': 'Name', 'start': 0, 'text': 'Name'},
 {'end': 24, 'entity': 'Name', 'start': 6, 'text': 'Md. Shariful Islam'}]

In [ ]:
import io
data1 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Akash kundu.pdf', 'rb').read())
resume_text1 = preprocess_data(data1)
entities1 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text1, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities1

[{'end': 5, 'entity': 'Name', 'start': 0, 'text': 'P E R'}]

In [ ]:
data3 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Akif Ahmed.pdf', 'rb').read())
resume_text3 = preprocess_data(data3)
entities3 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text3, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities3

[{'end': 129, 'entity': 'Name', 'start': 108, 'text': 'Syed Akif Ahmed Karim'}]

In [ ]:
data3 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Ariful Islam.pdf', 'rb').read())
resume_text3 = preprocess_data(data3)
entities3 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text3, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities3

[{'end': 129, 'entity': 'Name', 'start': 108, 'text': 'Syed Akif Ahmed Karim'}]

In [ ]:
data4 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Bidhan kumar Bhownick.pdf', 'rb').read())
resume_text4 = preprocess_data(data4)
entities4 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text4, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities4

[{'end': 12, 'entity': 'Name', 'start': 11, 'text': 'B'},
 {'end': 44, 'entity': 'Name', 'start': 23, 'text': 'BIDHAN KUMAR BHOWMICK'}]

In [ ]:
data5 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Diponkor Roy.pdf', 'rb').read())
resume_text5 = preprocess_data(data5)
entities5 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text5, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities5

[{'end': 12, 'entity': 'Name', 'start': 0, 'text': 'Diponkor Roy'},
 {'end': 224, 'entity': 'Name', 'start': 222, 'text': 'on'}]

In [ ]:
data6 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Jannatul Ferdous Meem.pdf', 'rb').read())
resume_text6 = preprocess_data(data6)
entities6 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text6, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities6

[{'end': 21, 'entity': 'Name', 'start': 0, 'text': 'Jannatul Ferdous[Meem'}]

In [ ]:
data7 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Kamrul Islam.pdf', 'rb').read())
resume_text7 = preprocess_data(data7)
entities7 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text7, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities7

[{'end': 508, 'entity': 'Name', 'start': 500, 'text': '. KAMRUL'},
 {'end': 1678, 'entity': 'Name', 'start': 1676, 'text': 'Ma'},
 {'end': 1685, 'entity': 'Name', 'start': 1682, 'text': 'Ash'}]

In [ ]:
data8 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/Kibtia Chowdhury.pdf', 'rb').read())
resume_text8 = preprocess_data(data8)
entities8 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text8, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities8

[{'end': 43, 'entity': 'Name', 'start': 27, 'text': 'KIBTIA CHOWDHURY'}]

In [ ]:
data9 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/MARZAHAN_SULTANA_CV_SQAT.pdf', 'rb').read())
resume_text9 = preprocess_data(data9)
entities9 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text9, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
entities9

[{'end': 24, 'entity': 'Name', 'start': 3, 'text': 'MARZAHAN SULTANA PILI'}]

In [ ]:
data10 = io.BytesIO(open('/content/drive/MyDrive/Applicants Resume/DevOps-IT-QA/MD. ABU MAS-UD SAYEED.pdf', 'rb').read())
resume_text10 = preprocess_data(data10)
entities10 = predict(model, TOKENIZER, idx2tag,
 DEVICE, resume_text10, MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
for i in entities9:
  if (len(i["text"])>10):
    print(i["text"])

MARZAHAN SULTANA PILI
